In [3]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image


In [4]:
# Hyperparameter
random_seed = 1
learning_rate = 0.001
batch_size = 128
num_epochs = 10

# Architechture
num_features = 28*28
num_classes = 10

# Other
grayscale = True

In [6]:
# Dataset
# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([128, 1, 28, 28])
Image label dimensions: torch.Size([128])


In [7]:
torch.manual_seed(0)

for epoch in range(2):
    for batch_idx, (x, y) in enumerate(train_loader):
        print('Epoch:', epoch+1, end="")
        print('|Batch index:', batch_idx, end="")
        print('|Batch size:', y.size()[0])
        break

Epoch: 1|Batch index: 0|Batch size: 128
Epoch: 2|Batch index: 0|Batch size: 128


In [9]:
# 3x3 convolution with padding
def conv3x3(in_planes, out_planes, stride=1):  #?
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1  #=4 in ResNet 50
    
    def __init__(self, inplanes, planes, stride=1, downsample=None):  #?downsample
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self,x):
        residual = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            residual = self.downsample(x)   #?
        
        out += residual
        out = self.relu(out)
        
        return out
            
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classses, grayscale): #layers: how many times to reuse the block eg. [3, 4, 6 , 3]
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3
        super(ResNet, self).__init__()
        # Initial layer
        self.conv1 = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3, bias=False)  
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # ResNet layers
        self.layer1 = self._make_layer(block, 64, layers[0]) #64*4 in the end
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)  # fix to a particular size eg 1x1
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2./n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                
    def _make_layer(self, block, planes, blocks, stride=1):  # residual/shortcut resize
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:  # for identity mapping
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,               
                            kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes*block.expansion)
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))  # change number of channels eg from 64 to 256
        self.inplanes = planes*block.expansion  # eg 256
        for i in range(1, blocks):  # = (# of block -1) because already one layer (layers.append(block(...)))
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)  #unpack the list
                                        # The output of each ResNet layer = 64 because 256 needed to be mapped back to 64, in the and out put 64*4 = 256 again
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # x = self.avgpool(x)  #disable because MNIST is already 1x1 #?
        
        x = x.view(x.size(0), -1)
        logits = self.fc(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas
    
    def resnet18(num_classes):
        model = ResNet(block=BasicBlock,
                       layers=[2, 2, 2, 2],
                       num_classes = num_classes,
                       grayscale = grayscale)
        return model
    
    torch.manual_seed(random_seed)
    model = resnet18(num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        

In [11]:
print([i for i in range(1,2)])


[1]
